In [1]:
%cd ..

/Users/Matteo/PycharmProjects/TrafficEmu


# Bayesian optimization: finding the minimum time loss

In this notebook we will performing bayesian optimization on the emulator trained on our sumo simulation. We are interested in identifying the parameters which enable us to obtain the minimum travel time. Our emulator is a gaussian process with an **rbf** kernel.


## Imports

In [2]:
import pickle
import numpy as np
import emukit as ek
import GPy
import pandas as pd

from emukit.model_wrappers import GPyModelWrapper
from emukit.core.initial_designs import RandomDesign
from emukit.core import ParameterSpace, ContinuousParameter, DiscreteParameter
from emukit.bayesian_optimization.acquisitions import ExpectedImprovement
from emukit.bayesian_optimization.loops import BayesianOptimizationLoop

from experimental_design import config
from sumo_grid_simulation.grid_simulation import Simulator

warning in stationary: failed to import cython module: falling back to numpy
warning in coregionalize: failed to import cython module: falling back to numpy
warning in choleskies: failed to import cython module: falling back to numpy


## Simulator

We start by initialising our simulation.

In [3]:
simulator = Simulator(end_time=300)

We then define the two user functions in which we are interested:

In [4]:
def user_function_time_loss(X):
    """  X = inputs - emukit doesnt pass named args, just an NxM ndarray, N is the number of points to evaluate, M is the number of parameters per each point """
    result = []
    i = 0
    
    print(X)
    print(f'\nUser function time loss called with {X.shape[0]} inputs to simulate')

    
    for gridSize, edgeMaxSpeed, maxSpeed, edgeLength, numLanes, accel in X:
        print(f'\nEvaluating input: {i+1} of {X.shape[0]}\n')
        
        alpha = 0.005
        max_number_of_vehicles = ((gridSize - 1) * gridSize * 2 + 4 * gridSize) * edgeLength / 5
        period = 300/(max_number_of_vehicles * alpha)
        
        s = simulator.simulate(
            gridSize      = int(gridSize),
            edgeMaxSpeed  = edgeMaxSpeed,
            maxSpeed      = maxSpeed,
            edgeLength    = edgeLength,
            numberOfLanes = int(numLanes),
            accel         = accel,
            trips_generator_period = period
        )
        # average time loss / average route duration
        result.append(s['timeLoss']/(s['duration'] * s['speed']))
        print(f'\nOutput {result[-1]}\n')
        i += 1
        
    # expand dims is essential or the acquition function breaks
    return np.expand_dims(np.array(result), 1)  

We then load the parameters' space from our configuration file:

In [5]:
parameter_space = config.get_parameter_space()

## Emulator

### Rnadomly initialised emulator

The first emulator we are going to analyse is the one that is just initilised with random points

We first sample at random 200 datapoints from the parameter space:

In [6]:
init_X, init_Y = pickle.load(open('bayesian_optimization_new_outputs/init_points/270_init_points_timeLoss.pkl', 'rb'))

'''
design = RandomDesign(parameter_space)
num_init_points = 270
init_X = design.get_samples(num_init_points)
init_Y = user_function_time_loss(init_X)
init_points = init_X, init_Y
with open(f'bayesian_optimization_new_outputs/init_points/{num_init_points}_init_points_timeLoss.pkl', "wb") as f:
     pickle.dump(init_points, f)
'''

[[15.         10.95530328  8.50135252 38.44477758  2.          3.23081874]
 [20.         11.84572713  5.54677651 64.4456652   1.          2.59573033]
 [ 3.         20.51174277 23.41318614 63.27962202  1.          2.31506699]
 ...
 [ 4.          8.26418169 21.27690617 37.34643847  1.          4.5032753 ]
 [11.          9.96123199 10.63811325 59.52706977  1.          4.98865182]
 [13.         22.71370816 18.43961177 31.28531633  3.          4.67061323]]

User function time loss called with 270 inputs to simulate

Evaluating input: 1 of 270

 Retrying in 1 seconds

Output 0.02322034411176497


Evaluating input: 2 of 270

 Retrying in 1 seconds

Output 0.02538874343082386


Evaluating input: 3 of 270

 Retrying in 1 seconds

Output 0.04944836317598119


Evaluating input: 4 of 270

 Retrying in 1 seconds

Output 0.02622778687564228


Evaluating input: 5 of 270

 Retrying in 1 seconds

Output 0.024874719610556963


Evaluating input: 6 of 270

 Retrying in 1 seconds

Output 0.0257540649794870

 Retrying in 1 seconds

Output 0.04745562130177514


Evaluating input: 95 of 270

 Retrying in 1 seconds

Output 0.028300652965339932


Evaluating input: 96 of 270

 Retrying in 1 seconds

Output 0.04106944240799794


Evaluating input: 97 of 270

 Retrying in 1 seconds

Output 0.0535351599923357


Evaluating input: 98 of 270

 Retrying in 1 seconds

Output 0.018702230403218906


Evaluating input: 99 of 270

 Retrying in 1 seconds

Output 0.05541029237640464


Evaluating input: 100 of 270

 Retrying in 1 seconds

Output 0.014940222133939092


Evaluating input: 101 of 270

 Retrying in 1 seconds

Output 0.026538044320471098


Evaluating input: 102 of 270

 Retrying in 1 seconds

Output 0.03596994353296874


Evaluating input: 103 of 270

 Retrying in 1 seconds

Output 0.03217702618711446


Evaluating input: 104 of 270

 Retrying in 1 seconds

Output 0.019937313028968295


Evaluating input: 105 of 270

 Retrying in 1 seconds

Output 0.023889098098286574


Evaluating input: 106 of 270

 Ret


Output 0.03476698334233765


Evaluating input: 193 of 270

 Retrying in 1 seconds

Output 0.025919525260763732


Evaluating input: 194 of 270

 Retrying in 1 seconds

Output 0.02538526939850842


Evaluating input: 195 of 270

 Retrying in 1 seconds

Output 0.07977794116913806


Evaluating input: 196 of 270

 Retrying in 1 seconds

Output 0.02053994033937329


Evaluating input: 197 of 270

 Retrying in 1 seconds

Output 0.053319945925371766


Evaluating input: 198 of 270

 Retrying in 1 seconds

Output 0.02138209373198254


Evaluating input: 199 of 270

 Retrying in 1 seconds

Output 0.030551184161955274


Evaluating input: 200 of 270

 Retrying in 1 seconds

Output 0.020230533992001882


Evaluating input: 201 of 270

 Retrying in 1 seconds

Output 0.019309973908225853


Evaluating input: 202 of 270

 Retrying in 1 seconds

Output 0.023544678828293347


Evaluating input: 203 of 270

 Retrying in 1 seconds

Output 0.03003184854653598


Evaluating input: 204 of 270

 Retrying in 1 second

To then fit a GP to these points

In [17]:
emulator = GPy.models.GPRegression(init_X, init_Y, noise_var=1e-10)
emukit_model_random_init = GPyModelWrapper(emulator, n_restarts=5)
emukit_model_random_init.optimize()

Optimization restart 1/5, f = -903.3391313189388
Optimization restart 2/5, f = -525.4434072102775
Optimization restart 3/5, f = -525.4433211153914
Optimization restart 4/5, f = -947.3283406818035
Optimization restart 5/5, f = -952.6877361143877


 /Users/Matteo/opt/anaconda3/envs/ml_and_physical/lib/python3.8/site-packages/GPy/kern/src/stationary.py:168: RuntimeWarning:overflow encountered in true_divide
 /Users/Matteo/opt/anaconda3/envs/ml_and_physical/lib/python3.8/site-packages/GPy/kern/src/rbf.py:76: RuntimeWarning:invalid value encountered in multiply


### Experimentally designed emulator with model variance

We also load the emulator obtained during experimental design:

In [18]:
emukit_model_variance = pickle.load(open('experimental_design_new_outputs/models/model_variance_20_init_points_250_loops_timeloss.pkl', 'rb'))

### Experimentally designed emulator with integrated variance reduction

We also load the emulator obtained during experimental design:

In [19]:
emukit_model_integrated_variance = pickle.load(open('experimental_design_new_outputs/models/integrated_variance_reduction_20_init_points_250_loops_timeloss.pkl', 'rb'))

## Bayesian optimisation

We now run bayesian optimisation on the emulators

In [20]:
n_iter_bo = 50

### Acquisition functions

In [21]:
acquisition_random_init = ExpectedImprovement(emukit_model_random_init)

In [22]:
acquisition_model_variance = ExpectedImprovement(emukit_model_variance)

In [23]:
acquisition_integrated_variance = ExpectedImprovement(emukit_model_integrated_variance)

### Optimisation loop

In [24]:
bo_random_init = BayesianOptimizationLoop(parameter_space, emukit_model_random_init, acquisition=acquisition_random_init)
bo_random_init.run_loop(user_function_time_loss, n_iter_bo)

Optimization restart 1/5, f = -973.8261538800683
Optimization restart 2/5, f = -977.563649452135
Optimization restart 3/5, f = -977.5636492255944
Optimization restart 4/5, f = -977.5636494511845
Optimization restart 5/5, f = -977.5636494513387
[[20.         8.        13.6151541 70.         1.         1.5      ]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds

Output 0.01360044346469686

Optimization restart 1/5, f = -981.5834967185365
Optimization restart 2/5, f = -527.8126394964941
Optimization restart 3/5, f = -981.5835473186708
Optimization restart 4/5, f = -527.8126396430891
Optimization restart 5/5, f = -981.5835473172617
[[14.  8. 27. 70.  1.  5.]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds

Output 0.02779356365745487

Optimization restart 1/5, f = -983.1552800890477
Optimization restart 2/5, f = -529.9360327571405
Optimization restart 3/5, f = -983.155280

 /Users/Matteo/opt/anaconda3/envs/ml_and_physical/lib/python3.8/site-packages/GPy/kern/src/stationary.py:168: RuntimeWarning:overflow encountered in true_divide
 /Users/Matteo/opt/anaconda3/envs/ml_and_physical/lib/python3.8/site-packages/GPy/kern/src/rbf.py:76: RuntimeWarning:invalid value encountered in multiply


Optimization restart 2/5, f = -962.6072988172517
Optimization restart 3/5, f = -997.3900046663828
Optimization restart 4/5, f = -541.4082487113559
Optimization restart 5/5, f = -997.3900046758081
[[14.         25.         10.05415025 70.          1.          5.        ]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds

Output 0.020851686490029404

Optimization restart 1/5, f = -1000.9691628416688


 /Users/Matteo/opt/anaconda3/envs/ml_and_physical/lib/python3.8/site-packages/GPy/kern/src/stationary.py:168: RuntimeWarning:overflow encountered in true_divide
 /Users/Matteo/opt/anaconda3/envs/ml_and_physical/lib/python3.8/site-packages/GPy/kern/src/rbf.py:76: RuntimeWarning:invalid value encountered in multiply


Optimization restart 2/5, f = -979.6896815274711
Optimization restart 3/5, f = -1000.9691628233073
Optimization restart 4/5, f = -1000.9691628809902
Optimization restart 5/5, f = -1000.9691628835546
[[20.          8.          9.96200744 70.          3.          1.5       ]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds

Output 0.014334135524863388

Optimization restart 1/5, f = -1005.1165262089899
Optimization restart 2/5, f = -546.0465266281547
Optimization restart 3/5, f = -1005.1165469617131
Optimization restart 4/5, f = -1005.1165469854939
Optimization restart 5/5, f = -946.7655861832034
[[20.          8.         11.75673425 66.73054915  1.          1.5       ]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds

Output 0.014367475275889273

Optimization restart 1/5, f = -1009.4076595311294
Optimization restart 2/5, f = -1009.4076594374754
Optimization restart 3/5,

...

 /Users/Matteo/opt/anaconda3/envs/ml_and_physical/lib/python3.8/site-packages/GPy/kern/src/stationary.py:168: RuntimeWarning:overflow encountered in true_divide
 /Users/Matteo/opt/anaconda3/envs/ml_and_physical/lib/python3.8/site-packages/GPy/kern/src/rbf.py:52: RuntimeWarning:overflow encountered in square
 /Users/Matteo/opt/anaconda3/envs/ml_and_physical/lib/python3.8/site-packages/GPy/kern/src/rbf.py:76: RuntimeWarning:invalid value encountered in multiply


Optimization restart 4/5, f = -1007.0539031361986
Optimization restart 5/5, f = -991.6929738631167
[[11.  25.   5.  30.   1.   1.5]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds

Output 0.02292618920594639

Optimization restart 1/5, f = -1047.1513199906424
Optimization restart 2/5, f = -994.7481838915962
Optimization restart 3/5, f = -994.7481840382933
Optimization restart 4/5, f = -994.6644648099028
Optimization restart 5/5, f = -994.7481840370648
[[20.          8.         11.68529041 66.75713828  1.          1.5       ]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds

Output 0.014630627229147986

Optimization restart 1/5, f = -1051.5357112078275
Optimization restart 2/5, f = -1051.5357111693195
Optimization restart 3/5, f = -997.3824779284184
Optimization restart 4/5, f = -997.3824779288555
Optimization restart 5/5, f = -999.9837652330895
[[20.         25.      

 /Users/Matteo/opt/anaconda3/envs/ml_and_physical/lib/python3.8/site-packages/GPy/kern/src/stationary.py:168: RuntimeWarning:overflow encountered in true_divide
 /Users/Matteo/opt/anaconda3/envs/ml_and_physical/lib/python3.8/site-packages/GPy/kern/src/rbf.py:52: RuntimeWarning:overflow encountered in square
 /Users/Matteo/opt/anaconda3/envs/ml_and_physical/lib/python3.8/site-packages/GPy/kern/src/rbf.py:76: RuntimeWarning:invalid value encountered in multiply


Optimization restart 3/5, f = -1066.2969382393505
Optimization restart 4/5, f = -1094.2719599797326
Optimization restart 5/5, f = -1050.2233459315637
[[20.          8.         11.77659687 67.77950506  1.          1.5       ]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds

Output 0.014182499838727426

Optimization restart 1/5, f = -1098.7231975479244
Optimization restart 2/5, f = -1098.7231975588033
Optimization restart 3/5, f = -1037.6722312821237
Optimization restart 4/5, f = -1057.0092327991379
Optimization restart 5/5, f = -1057.009232799744
[[11.          8.         10.38476761 52.15403523  1.          1.5       ]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds

Output 0.017621907022828683

Optimization restart 1/5, f = -1103.0134954919674
Optimization restart 2/5, f = -1103.013495492491
Optimization restart 3/5, f = -1103.0134954915227
Optimization restart 4/5

Optimization restart 2/5, f = -1173.3706631750827
Optimization restart 3/5, f = -1152.876881870478
Optimization restart 4/5, f = -1152.8768818704198
Optimization restart 5/5, f = -1152.8768818705564
[[20.         25.          6.9718012  65.66516483  1.          1.5       ]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds

Output 0.012831135219040779

Optimization restart 1/5, f = -1177.8583236952732
Optimization restart 2/5, f = -1159.1842269192962
Optimization restart 3/5, f = -1159.1842269219771
Optimization restart 4/5, f = -1159.1842269220379
Optimization restart 5/5, f = -1159.1842269179585
[[18.         25.          7.15380553 64.7531916   1.          1.5       ]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds

Output 0.013899978781177644

Optimization restart 1/5, f = -1182.369983816472
Optimization restart 2/5, f = -1162.1784729857643
Optimization restart 3/5

In [25]:
bo_model_variance = BayesianOptimizationLoop(parameter_space, emukit_model_variance, acquisition=acquisition_model_variance)
bo_model_variance.run_loop(user_function_time_loss, n_iter_bo)

Optimization restart 1/5, f = -845.0994970609856
Optimization restart 2/5, f = -472.4333852198935
Optimization restart 3/5, f = -845.0994970610072
Optimization restart 4/5, f = -472.43338641002845
Optimization restart 5/5, f = -845.0994970240088
[[15.          8.         11.45599919 56.21217159  1.          1.5       ]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds

Output 0.01556138757107769

Optimization restart 1/5, f = -848.0804474607623
Optimization restart 2/5, f = -474.6153809161699
Optimization restart 3/5, f = -848.0804474619067
Optimization restart 4/5, f = -848.0804474267142
Optimization restart 5/5, f = -474.6166820368311
[[14.          8.         11.37063869 59.78398119  3.          1.5       ]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds

Output 0.01908691650555003

Optimization restart 1/5, f = -850.7417755228048
Optimization restart 2/5, f = -476

 /Users/Matteo/opt/anaconda3/envs/ml_and_physical/lib/python3.8/site-packages/GPy/kern/src/stationary.py:168: RuntimeWarning:overflow encountered in true_divide
 /Users/Matteo/opt/anaconda3/envs/ml_and_physical/lib/python3.8/site-packages/GPy/kern/src/rbf.py:52: RuntimeWarning:overflow encountered in square
 /Users/Matteo/opt/anaconda3/envs/ml_and_physical/lib/python3.8/site-packages/GPy/kern/src/rbf.py:76: RuntimeWarning:invalid value encountered in multiply


Optimization restart 2/5, f = -855.4215253047369
Optimization restart 3/5, f = -879.7644987721171
Optimization restart 4/5, f = -879.764498772125
Optimization restart 5/5, f = -879.7644987718654
[[20.          8.         13.24138727 70.          3.          1.5       ]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds

Output 0.014334135524863388

Optimization restart 1/5, f = -883.6931928928007
Optimization restart 2/5, f = -883.693200219963
Optimization restart 3/5, f = -883.6932002188311
Optimization restart 4/5, f = -883.6932001802309


 /Users/Matteo/opt/anaconda3/envs/ml_and_physical/lib/python3.8/site-packages/GPy/kern/src/stationary.py:168: RuntimeWarning:overflow encountered in true_divide
 /Users/Matteo/opt/anaconda3/envs/ml_and_physical/lib/python3.8/site-packages/GPy/kern/src/rbf.py:52: RuntimeWarning:overflow encountered in square
 /Users/Matteo/opt/anaconda3/envs/ml_and_physical/lib/python3.8/site-packages/GPy/kern/src/rbf.py:76: RuntimeWarning:invalid value encountered in multiply


Optimization restart 5/5, f = -882.4742150072416
[[14.          8.         11.61933322 65.53556655  3.          1.5       ]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds

Output 0.01560455286431008

Optimization restart 1/5, f = -887.6938385345868
Optimization restart 2/5, f = -887.693838539479
Optimization restart 3/5, f = -887.6938385329165
Optimization restart 4/5, f = -887.6938385403089


 /Users/Matteo/opt/anaconda3/envs/ml_and_physical/lib/python3.8/site-packages/GPy/kern/src/stationary.py:168: RuntimeWarning:overflow encountered in true_divide
 /Users/Matteo/opt/anaconda3/envs/ml_and_physical/lib/python3.8/site-packages/GPy/kern/src/rbf.py:52: RuntimeWarning:overflow encountered in square
 /Users/Matteo/opt/anaconda3/envs/ml_and_physical/lib/python3.8/site-packages/GPy/kern/src/rbf.py:76: RuntimeWarning:invalid value encountered in multiply


Optimization restart 5/5, f = -861.6187227265883
[[20.          8.         13.16059578 70.          3.          1.5       ]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds

Output 0.014334135524863388

Optimization restart 1/5, f = -891.6808070566575
Optimization restart 2/5, f = -836.8830607387679
Optimization restart 3/5, f = -836.8830607375146
Optimization restart 4/5, f = -836.8830607342065
Optimization restart 5/5, f = -891.680998920048
[[18.          9.02598824 12.7520962  70.          1.          1.5       ]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds

Output 0.014710891976692066

Optimization restart 1/5, f = -895.6990529218841
Optimization restart 2/5, f = -895.6992029487385
Optimization restart 3/5, f = -830.9117918669216
Optimization restart 4/5, f = -830.9118309197042
Optimization restart 5/5, f = -830.911780802643
[[18.         10.44203614 12.109298

Optimization restart 5/5, f = -927.4382358429166
[[20.         8.        13.1642003 70.         3.         1.5      ]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds

Output 0.014334135524863388

Optimization restart 1/5, f = -963.7451354884499
Optimization restart 2/5, f = -941.238642527514
Optimization restart 3/5, f = -963.7451353589494
Optimization restart 4/5, f = -941.4414406665514
Optimization restart 5/5, f = -540.4929352591378
[[13.         12.16220146  9.95189008 64.76669324  1.          1.5       ]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds

Output 0.01799035383622415

Optimization restart 1/5, f = -967.7779355257504
Optimization restart 2/5, f = -941.3792364113866
Optimization restart 3/5, f = -941.3809612342793
Optimization restart 4/5, f = -947.2302227608803
Optimization restart 5/5, f = -941.3789923174529
[[20.          8.         13.80650455 70.

Optimization restart 4/5, f = -1033.2590330011867
Optimization restart 5/5, f = -1027.2111427125
[[18.          8.         12.86908286 70.          3.          1.5       ]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds

Output 0.013874790618092915

Optimization restart 1/5, f = -1037.4937728817324
Optimization restart 2/5, f = -1034.7308978084275
Optimization restart 3/5, f = -1037.4938332036027
Optimization restart 4/5, f = -1032.313125502198
Optimization restart 5/5, f = -1037.4938332012032
[[17.          8.         12.66760823 70.          3.          1.5       ]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds

Output 0.014077402369104007

Optimization restart 1/5, f = -1041.7281703561634
Optimization restart 2/5, f = -1042.4817184648703
Optimization restart 3/5, f = -1042.4817184639314
Optimization restart 4/5, f = -1040.173609992062
Optimization restart 5/5, f

In [26]:
bo_integrated_variance = BayesianOptimizationLoop(parameter_space, emukit_model_integrated_variance, acquisition=acquisition_integrated_variance)
bo_integrated_variance.run_loop(user_function_time_loss, n_iter_bo)

Optimization restart 1/5, f = -881.5250398990721
Optimization restart 2/5, f = -881.5250398988362
Optimization restart 3/5, f = -881.5250398980236
Optimization restart 4/5, f = -881.5250398989799
Optimization restart 5/5, f = -881.5250398986311
[[16.          8.         10.64972731 70.          1.          1.5       ]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds

Output 0.014104401186404993

Optimization restart 1/5, f = -885.3551307394091
Optimization restart 2/5, f = -491.6874975742553
Optimization restart 3/5, f = -885.3551307636716
Optimization restart 4/5, f = -885.355130764195
Optimization restart 5/5, f = -885.3551305455496
[[16.          8.         12.72765535 63.05674163  1.          1.5       ]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds

Output 0.014355979793665064

Optimization restart 1/5, f = -889.3029650205392
Optimization restart 2/5, f = -493

 /Users/Matteo/opt/anaconda3/envs/ml_and_physical/lib/python3.8/site-packages/GPy/kern/src/stationary.py:168: RuntimeWarning:overflow encountered in true_divide
 /Users/Matteo/opt/anaconda3/envs/ml_and_physical/lib/python3.8/site-packages/GPy/kern/src/rbf.py:76: RuntimeWarning:invalid value encountered in multiply


Optimization restart 4/5, f = -883.5992939290796
Optimization restart 5/5, f = -500.5225813893453
[[17.          8.         10.64280143 70.          1.          1.5       ]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds

Output 0.013049082122320577

Optimization restart 1/5, f = -903.8767164220696
Optimization restart 2/5, f = -903.8767163619048
Optimization restart 3/5, f = -776.9861395198587
Optimization restart 4/5, f = -903.8767164217527
Optimization restart 5/5, f = -903.8767164220596
[[15.          8.         13.33311733 59.56683977  1.          1.5       ]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds

Output 0.015766914721618557

Optimization restart 1/5, f = -907.7901902203636
Optimization restart 2/5, f = -907.7902573985662
Optimization restart 3/5, f = -780.8365611885447
Optimization restart 4/5, f = -907.7902574241172
Optimization restart 5/5, f = -50

 /Users/Matteo/opt/anaconda3/envs/ml_and_physical/lib/python3.8/site-packages/GPy/kern/src/rbf.py:52: RuntimeWarning:overflow encountered in square
 /Users/Matteo/opt/anaconda3/envs/ml_and_physical/lib/python3.8/site-packages/GPy/kern/src/stationary.py:168: RuntimeWarning:overflow encountered in true_divide
 /Users/Matteo/opt/anaconda3/envs/ml_and_physical/lib/python3.8/site-packages/GPy/kern/src/rbf.py:76: RuntimeWarning:invalid value encountered in multiply


Optimization restart 4/5, f = -924.5325374191815
Optimization restart 5/5, f = -927.2962417185603
[[14.          8.         27.         59.91162245  1.          5.        ]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds

Output 0.02828697168290859

Optimization restart 1/5, f = -929.8903773925514
Optimization restart 2/5, f = -799.2910386357544
Optimization restart 3/5, f = -929.8903773940914
Optimization restart 4/5, f = -929.8903773880118
Optimization restart 5/5, f = -929.8903774059596
[[16.          8.         12.79904446 60.22884869  1.          1.5       ]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds

Output 0.015448520680885331

Optimization restart 1/5, f = -933.901590344774
Optimization restart 2/5, f = -805.1614478054872
Optimization restart 3/5, f = -520.6015724894442
Optimization restart 4/5, f = -933.9015903908255
Optimization restart 5/5, f = -933.

 /Users/Matteo/opt/anaconda3/envs/ml_and_physical/lib/python3.8/site-packages/GPy/kern/src/stationary.py:168: RuntimeWarning:overflow encountered in true_divide
 /Users/Matteo/opt/anaconda3/envs/ml_and_physical/lib/python3.8/site-packages/GPy/kern/src/rbf.py:52: RuntimeWarning:overflow encountered in square
 /Users/Matteo/opt/anaconda3/envs/ml_and_physical/lib/python3.8/site-packages/GPy/kern/src/rbf.py:76: RuntimeWarning:invalid value encountered in multiply


Optimization restart 4/5, f = -977.7757873425751
Optimization restart 5/5, f = -884.6419666642602
[[17.          8.         11.50426418 66.32315312  1.          1.5       ]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds

Output 0.013420720028144423

Optimization restart 1/5, f = -986.6815751201704
Optimization restart 2/5, f = -889.3378935398576
Optimization restart 3/5, f = -889.338184232106
Optimization restart 4/5, f = -898.0512928269202
Optimization restart 5/5, f = -889.3379031486455
[[18.         8.        10.7360049 70.         1.         1.5      ]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds

Output 0.01305183442815754

Optimization restart 1/5, f = -990.8374534810171
Optimization restart 2/5, f = -905.5350148778983
Optimization restart 3/5, f = -897.4935556009575
Optimization restart 4/5, f = -990.8374534828799
Optimization restart 5/5, f = -990.837453

Optimization restart 3/5, f = -1057.6642508411524
Optimization restart 4/5, f = -1011.9327342800747
Optimization restart 5/5, f = -1057.6642506200426
[[18.          8.         11.2920618  66.37035735  1.          1.5       ]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds

Output 0.013664633529149343

Optimization restart 1/5, f = -1061.9071655460605
Optimization restart 2/5, f = -1061.9071655499633
Optimization restart 3/5, f = -1061.907165549579
Optimization restart 4/5, f = -1009.2907371472252
Optimization restart 5/5, f = -1013.7527784166741
[[18.          9.12499627 10.40668918 70.          1.          1.5       ]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds

Output 0.013465625764005165

Optimization restart 1/5, f = -1066.1605312701179
Optimization restart 2/5, f = -1066.1605312740955
Optimization restart 3/5, f = -1019.8789576280504
Optimization restart 4/

## Results

In [27]:
parameters_name = ['Size of the grid', 'Speed limit in the net (m/s)', 'Max vehicles speed (m/s)', 'Length of the roads (m)', 'Number of lanes', 'Accelleration (m/s^2)']

### Results on the random initialised emulator

In [39]:
results_random_init = bo_random_init.get_results()

In [40]:
results_random_init_df = pd.DataFrame(results_random_init.minimum_location.reshape(1,-1))
results_random_init_df.columns = parameters_name
results_random_init_df

,Size of the grid,Speed limit in the net (m/s),Max vehicles speed (m/s),Length of the roads (m),Number of lanes,Accelleration (m/s^2)
0,20.0,25.0,6.971801,65.665165,1.0,1.5


In [41]:
results_random_init.minimum_value

0.012831135219040779

### Results on the experimentally designed emulator with model variance

In [42]:
results_model_variance = bo_model_variance.get_results()

In [43]:
results_random_init_df = pd.DataFrame(results_model_variance.minimum_location.reshape(1,-1))
results_random_init_df.columns = parameters_name
results_random_init_df

,Size of the grid,Speed limit in the net (m/s),Max vehicles speed (m/s),Length of the roads (m),Number of lanes,Accelleration (m/s^2)
0,18.0,8.210003,18.028266,69.601346,1.0,1.652888


In [44]:
results_model_variance.minimum_value

0.013351357065580445

### Results on the experimentally designed emulator with integrated variance reduction

In [45]:
results_integrated_variance = bo_integrated_variance.get_results()

In [46]:
results_random_init_df = pd.DataFrame(results_integrated_variance.minimum_location.reshape(1,-1))
results_random_init_df.columns = parameters_name
results_random_init_df

,Size of the grid,Speed limit in the net (m/s),Max vehicles speed (m/s),Length of the roads (m),Number of lanes,Accelleration (m/s^2)
0,20.0,25.0,7.315849,70.0,1.0,1.5


In [47]:
results_integrated_variance.minimum_value

0.012724465498361932

## Save stuff

In [37]:
with open(f'bayesian_optimization_new_outputs/bayesian_opt_results/results_random_init_timeLoss.pkl', "wb") as f:
     pickle.dump(results_random_init, f)

with open(f'bayesian_optimization_new_outputs/bayesian_opt_results/results_model_variance_timeLoss.pkl', "wb") as f:
     pickle.dump(results_model_variance, f)
        
with open(f'bayesian_optimization_new_outputs/bayesian_opt_results/results_integrated_variance_timeLoss.pkl', "wb") as f:
     pickle.dump(results_integrated_variance, f)

In [38]:
with open(f'bayesian_optimization_new_outputs/models/270_random_init_50_bay_opt_timeLoss.pkl', "wb") as f:
     pickle.dump(emukit_model_random_init, f)

with open(f'bayesian_optimization_new_outputs/models/270_model_variance_50_bay_timeLoss.pkl', "wb") as f:
     pickle.dump(emukit_model_variance, f)
        
with open(f'bayesian_optimization_new_outputs/models/270_integrated_variance_50_bay_timeLoss.pkl', "wb") as f:
     pickle.dump(emukit_model_integrated_variance, f)